In [9]:
import sklearn as sk
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import seaborn as sns

<h1> Preproccessing </h1> 

In [10]:
#Fetching the dataset
news_groups = fetch_20newsgroups(subset='train',remove=(['headers', 'footers', 'quotes']),shuffle=True, random_state=42)

In [11]:
#intializing count vectorizer
count_vect = CountVectorizer()

In [12]:
#vectorizes our data set
X_train_counts = count_vect.fit_transform(news_groups.data)

In [13]:
#tf (term frequency) check each document and calculates for each word its frequency in the document 
#idf (inverse document frequency) accounts for the fact that 
#words that are in many documents are less useful and gives them lower weights making more accurate calculations ideally

In [14]:
#Here we are tranforming our vectors to frequencies
tfidf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
X_train_tf_idf = tf_transformer.transform(X_train_counts)
X_train_tf_idf.shape

NameError: name 'tf_transformer' is not defined

<h1> Logistic Regression Classifier </h1>

In [ ]:
#clf = (Classifier) 
#here we are fitting our logistic regression classifier 
clf = LogisticRegression(random_state=0,multi_class='auto').fit(X_train_tf_idf, news_groups.target)

In [ ]:
#two quick test cases 
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, news_groups.target_names[category]))



In [ ]:
#initializing a pipeline in order to make the vectorizer to tranformer to classifier easier to work with 
text_clf = Pipeline([
('vect', CountVectorizer()),
('tfidf', TfidfTransformer()),
('clf', LogisticRegression(random_state=0,multi_class='auto',solver='liblinear')),
])

In [ ]:
#fitting our pipeline with news_groups data 
text_clf.fit(news_groups.data, news_groups.target)

In [ ]:
#This block is testing our classifiers 
twenty_test = fetch_20newsgroups(subset='test',remove=(['headers', 'footers', 'quotes']),shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)


<h1> Decsion Tree Classifier </h1> 

In [3]:
#here we are fitting our Descion tree classifier 
clf = DecisionTreeClassifier(random_state=0).fit(X_train_tf_idf, news_groups.target)

NameError: name 'X_train_tf_idf' is not defined

<h1> Support Vector Machine Classifier </h1> 

<h1> Ada boost Classifier </h1> 

<h1> Random Forest Classifier </h1> 